## RNNS

Written by KN, Feb-2017

Based largely upon: https://github.com/pkmital/CADL/blob/master/session-5/lecture-5.ipynb

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib

In [2]:
f, _ = urllib.request.urlretrieve('https://www.gutenberg.org/cache/epub/11/pg11.txt', 'alice.txt')
with open(f, 'r') as fp:
    txt = fp.read()

In [3]:
set(txt)

{'\x00',
 '\x01',
 '\x02',
 '\x03',
 '\x04',
 '\x05',
 '\x06',
 '\x07',
 '\x08',
 '\t',
 '\n',
 '\x0b',
 '\x0c',
 '\r',
 '\x0e',
 '\x0f',
 '\x10',
 '\x11',
 '\x12',
 '\x13',
 '\x14',
 '\x15',
 '\x16',
 '\x17',
 '\x18',
 '\x19',
 '\x1a',
 '\x1b',
 '\x1c',
 '\x1d',
 '\x1e',
 '\x1f',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '\x7f',
 '\x80',
 '\x81',
 '\x82',
 '\x83',
 '\x84',
 '\x85',
 '\x86',
 '\x87',
 '\x88',
 '\x89',
 '\x8a',
 '\x8b',
 '\x8c',
 '\x8d',
 '\x8e',
 '\

In [4]:
vocab = list(set(txt))
len(txt), len(vocab)

(60422, 256)

In [5]:
encoder = dict(zip(vocab, range(len(vocab))))
decoder = dict(zip(range(len(vocab)), vocab))

In [6]:
for k in encoder.keys():
    print('{0}:{1}'.format(k,encoder[k]))

 :0
�:1
:2
�:3
:4
�:5
:6
�:7
:8
�:9
:10
�:11
:12
�:13
:14
�:15
 :16
�:17
$:18
�:19
(:20
�:21
,:22
�:23
0:24
�:25
4:26
�:27
8:28
�:29
<:30
�:31
@:32
�:33
D:34
�:35
H:36
�:37
L:38
�:39
P:40
�:41
T:42
�:43
X:44
�:45
\:46
�:47
`:48
�:49
d:50
�:51
h:52
�:53
l:54
�:55
p:56
�:57
t:58
�:59
x:60
�:61
|:62
�:63
�:64
:65
�:66
:67
�:68
:69
�:70
:71
�:72
:73
�:74
:75
�:76
:77
�:78
:79
�:80
#:81
�:82
':83
�:84
+:85
�:86
/:87
�:88
3:89
�:90
7:91
�:92
;:93
�:94
?:95
�:96
C:97
�:98
G:99
�:100
K:101
�:102
O:103
�:104
S:105
�:106
W:107
�:108
[:109
�:110
_:111
�:112
c:113
�:114
g:115
�:116
k:117
�:118
o:119
�:120
s:121
�:122
w:123
�:124
{:125
�:126
:127
�:128
:129
�:130
:131
�:132

:133
�:134
:135
�:136
:137
�:138
:139
�:140
:141
�:142
:143
�:144
":145
�:146
&:147
�:148
*:149
�:150
.:151
�:152
2:153
�:154
6:155
�:156
::157
�:158
>:159
�:160
B:161
�:162
F:163
�:164
J:165
�:166
N:167
�:168
R:169
�:170
V:171
�:172
Z:173
�:174
^:175
�:176
b:177
�:178
f:179
�:180
j:181
�:182
n:183
�:184


In [7]:
# Define the model params
# Number of sequences in a mini batch
batch_size = 100

# Number of characters in a sequence
sequence_length = 100

# Number of cells in LSTM layer
n_cells = 256

# Number of LSTM layers
n_layers = 2

# Total number of characters in the one-hot encoding
n_chars = len(vocab)

In [8]:
# Inputs: batch size x size of sequence length
X = tf.placeholder(tf.int32, [None, sequence_length], name='X')

# Outputs
Y = tf.placeholder(tf.int32, [None, sequence_length], name='Y')

In [9]:
# Convert labels into embeddings through TF functionality:
# (vs. making one-hot encodings ourselves)

# Variable to go from one-hot representation to our LSTM cells
embedding = tf.get_variable("embedding", [n_chars, n_cells])

# Use tensorflow's embedding lookup to look up the ids in X
Xs = tf.nn.embedding_lookup(embedding, X)

# Can see: resulting lookups are concatenated into a dense tensor
print(Xs.get_shape().as_list())

[None, 100, 256]


In [ ]:
tf.split?
# tf.split(split_dim, num_split, value)
# splits (value) along (split_dim) into (num_split) tensors [must divide out equally]

In [ ]:
tf.squeeze?
# tf.squeeze(input, squeeze_dims) --> 
# returns same tensor with all dimensions of size 1 listed in (squeeze_dims) removed

In [13]:
# Need to make slices of sequences
# Let's create a name scope for the operations to clean things up in our graph
print(tf.split(1, sequence_length, Xs)[0])
with tf.name_scope('reslice'):
    Xs_list = [tf.squeeze(seq, [1]) for seq in tf.split(1, sequence_length, Xs)]  
print(Xs_list[0].get_shape().as_list())
print(len(Xs_list))

Tensor("split_3:0", shape=(?, 1, 256), dtype=float32)
[None, 256]
100


In [16]:
# create single recurrent layer
cells_single = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_cells, state_is_tuple=True)

In [17]:
# Initialize state
initial_state_single = cells_single.zero_state(tf.shape(X)[0], tf.float32)

In [18]:
# For multiple layers (we have specified multiple layers here)
cells = tf.nn.rnn_cell.MultiRNNCell([cells_single] * n_layers, state_is_tuple=True)
initial_state = cells.zero_state(tf.shape(X)[0], tf.float32)

In [ ]:
# Get list of outputs of every element in input sequence.

# Output will be batch_size x n_cells
# state will be (n_cells' memory, output)
outputs, state = tf.nn.rnn(cells, Xs_list, initial_state=initial_state)

# Stack all outputs for every cell
outputs_flat = tf.reshape(tf.concat(1, outputs), [-1, n_cells])

In [ ]:
# Do prediction step
with tf.variable_scope('prediction'):
    W = tf.get_variable(
        "W",
        shape=[n_cells, n_chars],
        initializer=tf.random_normal_initializer(stddev=0.1))
    b = tf.get_variable(
        "b",
        shape=[n_chars],
        initializer=tf.random_normal_initializer(stddev=0.1))

    # Find the output prediction of every single character in our minibatch
    # we denote the pre-activation prediction, logits.
    logits = tf.matmul(outputs_flat, W) + b

    # We get the probabilistic version by calculating the softmax of this
    probs = tf.nn.softmax(logits)

    # And then we can find the index of maximum probability
    Y_pred = tf.argmax(probs, 1)

In [ ]:
# Make loss function
with tf.variable_scope('loss'):
    # Compute mean cross entropy loss for each output.
    Y_true_flat = tf.reshape(tf.concat(1, Y), [-1])   # why add the 1??
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, Y_true_flat)
    mean_loss = tf.reduce_mean(loss)

In [ ]:
# clip the gradient
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    gradients = []
    clip = tf.constant(5.0, name="clip")
    for grad, var in optimizer.compute_gradients(mean_loss):
        gradients.append((tf.clip_by_value(grad, -clip, clip), var))
    updates = optimizer.apply_gradients(gradients)

In [ ]:
# train!
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

cursor = 0
it_i = 0
while True:
    Xs, Ys = [], []
    for batch_i in range(batch_size):
        if (cursor + sequence_length) >= len(txt) - sequence_length - 1:
            cursor = 0
        Xs.append([encoder[ch]
                   for ch in txt[cursor:cursor + sequence_length]])
        Ys.append([encoder[ch]
                   for ch in txt[cursor + 1: cursor + sequence_length + 1]])

        cursor = (cursor + sequence_length)
    Xs = np.array(Xs).astype(np.int32)
    Ys = np.array(Ys).astype(np.int32)

    loss_val, _ = sess.run([mean_loss, updates],
                           feed_dict={X: Xs, Y: Ys})
    print(it_i, loss_val)

    if it_i % 500 == 0:
        p = sess.run([Y_pred], feed_dict={X: Xs})[0]
        preds = [decoder[p_i] for p_i in p]
        print("".join(preds).split('\n'))

    it_i += 1